# Prepare script for relax_liquid calculations

This Notebook is designed to prepare (and run) relax_liquid calculations at progressively smaller temperatures.   


In [1]:
import iprPy
import numpy as np
import atomman as am
import potentials

In [2]:
database_name = 'master'
run_directory = 'master_4'
lammps_command = 'C:/Users/lmh1/Documents/LAMMPS/2022-09-15/bin/lmp.exe'

potential_LAMMPS_id = '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3'

In [3]:
database = iprPy.load_database(database_name)
potdb = potentials.Database(local_database=database, local=True, remote=False)

## Prepare and run at melt temperature

In [4]:
Tmelt = 3000

# Fetch relaxed_crystal fcc records
parents, parent_df = database.get_records('relaxed_crystal', family='A1--Cu--fcc', standing='good',
                                          method='dynamic', potential_LAMMPS_id=potential_LAMMPS_id,
                                          return_df=True)

parent_keys = parent_df.key.tolist()

database.master_prepare(
    styles = 'relax_liquid',
    np_per_runner = 1,
    run_directory = run_directory,
    num_pots = 900,
    lammps_command = lammps_command,
    temperature = str(Tmelt),
    temperature_melt = str(Tmelt),
    parent_key = parent_keys)

database.runner(run_directory)

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:08<00:00, 51.73it/s]


794 potential ids found
Preparing calculation relax_liquid branch main
Using potential #s 0 to 793

123 existing calculation records found
1029 matching interatomic potentials found
1 matching atomic parents found
1 calculation combinations to check
1 new records to prepare


preparing: 100%|#########################################################################| 1/1 [00:03<00:00,  3.06s/it]


Runner started with pid 19348


ec8dbd71-a734-41f2-83ef-8097fb0e3197
sim calculated successfully

No simulations left to run


## Load and check results

In [ ]:
calckey = '6ef01496-133d-401a-bc49-f152a64308ef'
tar = database.get_tar(name=calckey, style='calculation_relax_liquid')
f = tar.extractfile(f'{calckey}/log.lammps')
output = am.lammps.Log(f)
#content = f.readlines()

In [ ]:
thermo = output.simulations[1].thermo
plt.plot(thermo.Step, thermo.Volume)

In [ ]:
#content[-2]

## Prepare and run at lower T

In [6]:
for temperature in range(Tmelt-50, 0, -50):
    print(temperature)
    last_temp = temperature + 50
    parents, parent_df = database.get_records('calculation_relax_liquid', status='finished',
                                              potential_LAMMPS_id=potential_LAMMPS_id,
                                              return_df=True)
    parents = parents[parent_df.temperature == last_temp]
    parent_df = parent_df[parent_df.temperature == last_temp].reset_index(drop=True)
    if len(parent_df) == 0:
        print('No records found for the previous temperature')
        break
        
    # Construct global single keys
    prepare_keys = {}
    prepare_keys['lammps_command'] = lammps_command
    prepare_keys['sizemults'] = '1 1 1'
    prepare_keys['temperature'] = str(temperature)
    prepare_keys['temperature_melt'] = str(last_temp)
    prepare_keys['meltsteps'] = '0'

    # Initialize multi keys
    prepare_keys['potential_file'] = []
    prepare_keys['potential_content'] = []
    prepare_keys['potential_dir'] = []
    prepare_keys['potential_dir_content'] = []
    prepare_keys['potential_kim_id'] = []
    prepare_keys['potential_kim_potid'] = []
    prepare_keys['load_file'] = []
    prepare_keys['load_content'] = []
    prepare_keys['load_style'] = []
    prepare_keys['symbols'] = []
    prepare_keys['rdfcutoff'] = []

    for potential_LAMMPS_key in np.unique(parent_df.potential_LAMMPS_key):
        lmppot = potdb.get_lammps_potential(key=potential_LAMMPS_key)

        for i in parent_df[parent_df.potential_LAMMPS_key == potential_LAMMPS_key].index:
            parent = parents[i]
            parent_series = parent_df.loc[i]

            # Set potential info
            prepare_keys['potential_file'].append(f'{lmppot.name}.json')
            prepare_keys['potential_content'].append(f'record {lmppot.name}')
            if lmppot.pair_style != 'kim':
                prepare_keys['potential_dir'].append(lmppot.name)
                prepare_keys['potential_dir_content'].append(f'tar {lmppot.name}')
                prepare_keys['potential_kim_id'].append('')
                prepare_keys['potential_kim_potid'].append('')
            else:
                prepare_keys['potential_dir'].append('')
                prepare_keys['potential_dir_content'].append('')
                prepare_keys['potential_kim_id'].append(lmppot.id)
                if len(lmppot.potids) > 1:
                    prepare_keys['potential_kim_potid'].append(lmppot.potid)
                else:
                    prepare_keys['potential_kim_potid'].append('')

            # Set load file info
            prepare_keys['load_file'].append(f'{parent.name}/{parent.final_dump["filename"]}')
            prepare_keys['load_content'].append(f'tarfile {parent.name} {parent.final_dump["filename"]}')
            prepare_keys['load_style'].append('atom_dump')
            prepare_keys['symbols'].append(' '.join(parent.final_dump["symbols"]))
            
            # Set other info
            prepare_keys['rdfcutoff'].append(str(parent_series.rdfcutoff))
    
    database.prepare(run_directory, 'relax_liquid', **prepare_keys)
    database.runner(run_directory)

2950
303 existing calculation records found
1 calculation combinations to check
0 new records to prepare


preparing: 0it [00:00, ?it/s]

Runner started with pid 19348
No simulations left to run
2900


303 existing calculation records found
1 calculation combinations to check
0 new records to prepare


preparing: 0it [00:00, ?it/s]

...

preparing: 0it [00:00, ?it/s]

Runner started with pid 19348
No simulations left to run
50


303 existing calculation records found
1 calculation combinations to check
0 new records to prepare


preparing: 0it [00:00, ?it/s]

Runner started with pid 19348
No simulations left to run


## Check errors (if needed)

In [ ]:
calcname = '2a90ed0b-b982-4991-b6fa-b3f89b15a134'
tar = database.get_tar(name=calcname, style='calculation_relax_liquid')
f = tar.extractfile(f'{calcname}/log.lammps')
content = f.readlines()

In [ ]:
content[-2]